In [3]:
# Add this at the beginning of the notebook, before any other imports
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('.')))

In [9]:
from sqlalchemy import select, exists

from news_analyst_agent.db.database import AsyncSessionLocal
from news_analyst_agent.db.models import User, Thread, Step, Element, Feedback

In [6]:
async with AsyncSessionLocal() as session:
    query = select(Thread).order_by(Thread.createdAt.desc())
    results = await session.execute(query)
    threads = results.scalars().all()

2025-02-14 07:36:12,844 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-14 07:36:12,847 INFO sqlalchemy.engine.Engine SELECT threads.id, threads."createdAt", threads.name, threads."userId", threads."userIdentifier", threads.tags, threads.metadata 
FROM threads ORDER BY threads."createdAt" DESC
2025-02-14 07:36:12,847 INFO sqlalchemy.engine.Engine [cached since 12.96s ago] ()
2025-02-14 07:36:12,855 INFO sqlalchemy.engine.Engine ROLLBACK


In [13]:
async with AsyncSessionLocal() as session:
    # Using NOT EXISTS
    subquery = select(Step).where(Step.threadId == Thread.id)
    query = select(Thread).where(~exists(subquery)).order_by(Thread.createdAt.desc())
    
    result = await session.execute(query)
    threads_without_steps = result.scalars().all()


2025-02-14 07:39:43,175 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-14 07:39:43,177 INFO sqlalchemy.engine.Engine SELECT threads.id, threads."createdAt", threads.name, threads."userId", threads."userIdentifier", threads.tags, threads.metadata 
FROM threads 
WHERE NOT (EXISTS (SELECT steps.id, steps.name, steps.type, steps."threadId", steps."parentId", steps."disableFeedback", steps.streaming, steps."waitForAnswer", steps."isError", steps.metadata, steps.tags, steps.input, steps.output, steps."createdAt", steps.start, steps."end", steps.generation, steps."showInput", steps.language, steps.indent 
FROM steps 
WHERE steps."threadId" = threads.id)) ORDER BY threads."createdAt" DESC
2025-02-14 07:39:43,178 INFO sqlalchemy.engine.Engine [cached since 47.53s ago] ()
2025-02-14 07:39:43,185 INFO sqlalchemy.engine.Engine ROLLBACK


In [14]:
len(threads_without_steps)

17

In [15]:
async with AsyncSessionLocal() as session:
    # Using NOT EXISTS
    subquery = select(Step).where(Step.threadId == Thread.id)
    query = select(Thread).where(exists(subquery)).order_by(Thread.createdAt.desc())
    
    result = await session.execute(query)
    threads_with_steps = result.scalars().all()


2025-02-14 07:39:49,641 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-14 07:39:49,649 INFO sqlalchemy.engine.Engine SELECT threads.id, threads."createdAt", threads.name, threads."userId", threads."userIdentifier", threads.tags, threads.metadata 
FROM threads 
WHERE EXISTS (SELECT steps.id, steps.name, steps.type, steps."threadId", steps."parentId", steps."disableFeedback", steps.streaming, steps."waitForAnswer", steps."isError", steps.metadata, steps.tags, steps.input, steps.output, steps."createdAt", steps.start, steps."end", steps.generation, steps."showInput", steps.language, steps.indent 
FROM steps 
WHERE steps."threadId" = threads.id) ORDER BY threads."createdAt" DESC
2025-02-14 07:39:49,650 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ()
2025-02-14 07:39:49,667 INFO sqlalchemy.engine.Engine ROLLBACK


In [16]:
len(threads_with_steps)

4

In [22]:
thread_ids = [thread.id for thread in threads_with_steps]

In [23]:
thread_ids[0]


UUID('c81453a2-684e-4ab3-b5e3-c2fd4aa55352')

In [24]:
async with AsyncSessionLocal() as session:
    # Using NOT EXISTS
    query = select(Step).where(Step.threadId == Thread.id).order_by(Step.createdAt.desc())
    
    result = await session.execute(query)
    steps = result.scalars().all()

2025-02-14 07:52:34,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-14 07:52:34,539 INFO sqlalchemy.engine.Engine SELECT steps.id, steps.name, steps.type, steps."threadId", steps."parentId", steps."disableFeedback", steps.streaming, steps."waitForAnswer", steps."isError", steps.metadata, steps.tags, steps.input, steps.output, steps."createdAt", steps.start, steps."end", steps.generation, steps."showInput", steps.language, steps.indent 
FROM steps, threads 
WHERE steps."threadId" = threads.id ORDER BY steps."createdAt" DESC
2025-02-14 07:52:34,539 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ()
2025-02-14 07:52:34,552 INFO sqlalchemy.engine.Engine ROLLBACK


In [28]:
news = []
msg = []
for step in steps:
    if step.input == 'Retrieving news':
        news.append(step.output)
    msg.append({
        "input": step.input,
        "output": step.output,
    })

In [31]:
import json

json.loads(news[0])

[{'title': 'Microsoft Stock Is Falling as Artificial Intelligence (AI) Growth Is Getting More Expensive',
  'description': "To its credit, Microsoft is in a solid competitive position to benefit from enterprise investment in AI services. Microsoft's cloud revenue surpassed $40 billion and grew 21% year over year. AI-related revenue is up to $13 billion on an annualized basis, and up 175% year over year."},
 {'title': "You thought Magnificent 7 and tech stocks were the only winners of AI boom? Wait! Here's a sector whose stocks could be an under-the-radar winner, say analysts",
  'description': 'Retail stores such as Walmart and Target stand to gain from the enormous artificial intelligence investments being made by technology giants, without having to spend billions on infrastructure. According to Morgan Stanley analysts,'},
 {'title': "What DeepSeek's AI Innovation Means For Investors And Big Tech",
  'description': "DeepSeek's AI breakthrough challenges Big Tech with a cheaper, effic

In [27]:
msg

[{'input': None,
  'output': 'Here are some recent news articles related to DeepSeek:\n\n1. **DeepSeek Got The Spotlight, Alibaba Won Apple\'s AI Business**  \n   Apple has chosen Alibaba over DeepSeek to implement its AI-powered services for the iPhone in China, aiming to counter declining sales.\n\n2. **Elon Musk says Grok 3 will outperform ChatGPT, DeepSeek in the coming weeks**  \n   Elon Musk announced that his AI chatbot, Grok 3, is nearing completion and will soon be available, claiming it will outperform both ChatGPT and DeepSeek.\n\n3. **How Did DeepSeek Build Its A.I. With Less Money?**  \n   DeepSeek, a Chinese start-up, utilized various technological tricks, including a method called "mixture of experts," to significantly reduce the costs associated with building its AI technology.\n\n4. **What DeepSeek\'s Launch Means For The Human-in-the-Loop AI Market**  \n   DeepSeek\'s AI model is challenging traditional human-in-the-loop approaches by using synthetic data and expert i